In [1]:
import asyncio
import time
import json
import argparse
from pathlib import Path
from openai import AsyncOpenAI

In [2]:
# === Настройки ===
BASE_URL = "http://80.209.242.40:8000/v1"
API_KEY = "dummy-key"
MODEL_NAME = "llama-3.3-70b-instruct"
MAX_CONCURRENT_REQUESTS = 50  # Примерно 3000 в минуту
REQUESTS_PER_MINUTE = 3000
DELAY_BETWEEN_BATCHES = 60 / (REQUESTS_PER_MINUTE / MAX_CONCURRENT_REQUESTS)

# === Отправка одного запроса ===
async def send_request(client, content, semaphore, idx):
    async with semaphore:
        await asyncio.sleep(DELAY_BETWEEN_BATCHES)  # троттлинг

        try:
            start_time = time.time()
            response = await client.chat.completions.create(
                model=MODEL_NAME,
                messages=[{"role": "user", "content": content}],
                max_tokens=256,
                temperature=0.5,
            )
            latency = time.time() - start_time
            return {
                "id": idx,
                "content": content,
                "response": response.choices[0].message.content,
                "latency": latency,
            }
        except Exception as e:
            return {
                "id": idx,
                "content": content,
                "response": None,
                "error": str(e),
            }

# === Главная функция ===
async def main(input_path):
    input_path = Path(input_path)
    output_path = input_path.with_name(input_path.stem + "_responses.jsonl")

    # Загружаем все строки из JSONL
    with open(input_path, 'r', encoding='utf-8') as f:
        lines = [json.loads(line) for line in f]

    prompts = [line["prompt"] if "prompt" in line else line["content"] for line in lines]

    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)
    client = AsyncOpenAI(base_url=BASE_URL, api_key=API_KEY)

    tasks = [
        send_request(client, prompt, semaphore, idx)
        for idx, prompt in enumerate(prompts)
    ]

    # Обрабатываем ответы с сохранением по ходу
    with open(output_path, 'w', encoding='utf-8') as out_file:
        for future in asyncio.as_completed(tasks):
            result = await future
            out_file.write(json.dumps(result, ensure_ascii=False) + '\n')
            print(f"Processed #{result['id']}: {result['response'][:80] if result['response'] else 'ERROR'}")

    print(f"\nAll responses saved to: {output_path}")

# === Точка входа ===
if __name__ == "__main__":
    #parser = argparse.ArgumentParser(description="Run batch LLM requests.")
    #parser.add_argument("input_file", help="Path to .jsonl file with prompts")
    #args = parser.parse_args()

    #asyncio.run(main(args.input_file))
    asyncio.run(main('batch_chunks_miniml/attempt.jsonl'))

RuntimeError: asyncio.run() cannot be called from a running event loop